Загрузка и обработка файлов

In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from keras.applications import VGG16
from keras.applications import ResNet50

from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from sklearn.model_selection import KFold

In [2]:
# Загружаем данные из файлов CSV
train_data = pd.read_csv("train_data.csv")
test_data = pd.read_csv("test_data.csv")

In [3]:
# Разделяем данные на признаки (X) и метки (y)
X_train = train_data['image_path'].values
y_train = train_data['label'].values
X_test = test_data['image_path'].values
y_test = test_data['label'].values

# Выводим размеры обучающей, валидационной и тестовой выборок
print("Размер обучающей выборки:", len(X_train))
print("Размер тестовой выборки:", len(X_test))


Размер обучающей выборки: 6194
Размер тестовой выборки: 1554


In [4]:
# Сохраняем пути к изображениям в переменные
train_image_paths = X_train.tolist()
test_image_paths = X_test.tolist()

In [6]:

# Загрузка предобученной модели ResNet50
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Извлечение признаков из изображений
def extract_features(image_paths):
    features = []
    for path in image_paths:
        img = image.load_img(path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        feature = resnet_model.predict(x)
        features.append(feature.flatten())
    return np.array(features)

# Извлечение признаков из обучающих и тестовых изображений
X_train = extract_features(train_image_paths)
X_test = extract_features(test_image_paths)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━

In [32]:
# Инициализация и обучение классификатора KNeighborsClassifier
knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(X_train, y_train)

KNeighborsClassifier()

In [33]:
# Прогнозирование на тестовых данных
predictions = knn_classifier.predict(X_test)

In [34]:
# Создадим DataFrame с предсказаниями и фактическими значениями
predictions_df = pd.DataFrame({'Predicted': predictions, 'Actual': y_test})

# Добавим столбец 'Correct', содержащий информацию о совпадении предсказаний с фактическими значениями
predictions_df['Correct'] = predictions_df['Predicted'] == predictions_df['Actual']

# Сгруппируем таблицу по столбцу 'Correct' и подсчитаем количество совпадений и несовпадений для каждого значения
grouped = predictions_df.groupby('Correct').size()

# Выведем количество совпадений и несовпадений для каждого значения столбца 'Correct'
print(grouped)

Correct
False    1011
True      543
dtype: int64


In [35]:
# Рассчитаем стандартные метрики
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
conf_matrix = confusion_matrix(y_test, predictions)

C:\Users\14488\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [36]:
# Создадим датасет с метриками
metrics_data = {
    'Accuracy': [accuracy],
    'Precision': [precision],
    'Recall': [recall],
    'F1-Score': [f1]
}

In [37]:
# Создадим датафрейм с метриками
metrics_df = pd.DataFrame(metrics_data)

# Преобразуем матрицу ошибок в датафрейм
conf_matrix_df = pd.DataFrame(conf_matrix, columns=['Predicted ' + str(label) for label in range(conf_matrix.shape[1])], index=['Actual ' + str(label) for label in range(conf_matrix.shape[0])])

# Добавим матрицу ошибок к датасету с метриками
metrics_df['Confusion Matrix'] = [conf_matrix_df]

In [38]:
from termcolor import colored

# Создадим строку для заголовка
header = colored("Стандартные метрики модели и матрица ошибок:", 'blue')

# Выведем заголовок
print(header)

# Выведем метрики
print("\nСтандартные метрики:")
print(metrics_df)

# Выведем матрицу ошибок
print("\nМатрица ошибок:")
print(conf_matrix_df)

Стандартные метрики модели и матрица ошибок:

Стандартные метрики:
   Accuracy  Precision    Recall  F1-Score  \
0  0.349421   0.377315  0.305378  0.284237   

                                    Confusion Matrix  
0            Predicted 0  Predicted 1  Predicted ...  

Матрица ошибок:
          Predicted 0  Predicted 1  Predicted 2  Predicted 3  Predicted 4  \
Actual 0           96            3           46           27            7   
Actual 1            1           54           20            3            0   
Actual 2            4            0          168            7            1   
Actual 3            6            4          124          114            8   
Actual 4            5            4           28           21            9   
Actual 5           10            6           45           10            0   
Actual 6            3            4           99           18            4   
Actual 7           15            4           39           39           14   
Actual 8            